In [27]:
import os
import librosa
import soundfile as sf
from glob import glob
import datetime
import time

def create_directory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Failed to create the directory")

#---------------------------------------------------------wav파일 처리---------------------------------------------------------
def trim_audio(audio, sec_first_time, sec_last_time):
    sr = 16000 #samplimg rate
        
    y, sr = librosa.load(audio, sr=sr)
    
    ny = y[sr * sec_first_time : sr * sec_last_time]
    
    #save_file = audio[:-4] + "_" + str(sec_first_time)
    #save_file = save_path + '/' + audio_name[:-4] + "_" + str(count)
    
    #파일 이름과 경로 알아내기
    audio_name = os.path.basename(audio)
    file_directory = os.path.dirname(audio)
    parent_directory = os.path.dirname(file_directory)
    
    #파일 저장
    save_directory = parent_directory + '/save/' + audio_name[:-4]
    create_directory(save_directory)
    save_file = save_directory +'/' + audio_name[:-4] + '_' + str(sec_first_time) + '.wav'
    sf.write(save_file, ny, sr)
    
    return save_file
    


#---------------------------------------------------------txt파일 처리---------------------------------------------------------
def trim_text(i, text):
    #global sec_first_time, sec_last_time, first_time, last_time
    global last_time
    
    #시작 시간 (HH:MM:SS)
    first_time = text.split('\t')[0]
    
    #STT결과 Text
    label = text.split('\t')[1].replace('\n','')
    
    #끝나는 시간 (HH:MM:SS)
    #last_time = text_info[-len(text_info) + i + 1].split('\t')[0]
    if i == len(text_info)-1:
        # 마지막 구간 처리 필요
        pass
    else:
        last_time = text_info[i+1].split('\t')[0]
        
    #시작시간(first_time)과 끝나는시간(last_time)을 초단위로 표현
    if i == 0:
        x = time.strptime(first_time,"\ufeff%H:%M:%S")
        y = time.strptime(last_time,"%H:%M:%S")
    else:
        x = time.strptime(first_time,"%H:%M:%S")
        y = time.strptime(last_time,"%H:%M:%S")


    sec_first_time = int(datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds())
    sec_last_time = int(datetime.timedelta(hours=y.tm_hour,minutes=y.tm_min,seconds=y.tm_sec).total_seconds())
    
    return sec_first_time, sec_last_time, label

#---------------------------------------------------------main---------------------------------------------------------

base_path = 'C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727'
#base_path = 'C:/Users/USER/Desktop/전사 파일 작업 완료/2022_07_21_Vrew_완료_한승연/2022_07_21_Vrew_완료_한승연'
os.chdir(base_path) # base_path로 작업 디렉토리 이동

audio_path = base_path + '/audio'
text_path = base_path + '/text'
save_path = base_path + '/save'

#audio_list = os.listdir(audio_path)
audio_list = glob(audio_path +'/*.wav') #경로가 포함된 .wav파일 list
text_list = glob(text_path + '/*.txt') #경로가 포함된 .txt파일 list
#print(audio_list)
#print(text_list)

transcript_list = []
fname_list = []
data_list = []


for audio, text in zip(audio_list, text_list): 
    data_dic = {}
    print("audio : " + audio)
    print("text : " + text)
    print("="*110)
    
    with open(text, 'r', encoding='utf-8') as f:
        text_info = f.readlines()
        for i, text in enumerate(text_info):
            audio_data_dic = {}
            
            sec_first_time, sec_last_time, label = trim_text(i, text)
            audio_data_dic["transcript"] = label

            save_file = trim_audio(audio, sec_first_time, sec_last_time)
            audio_data_dic["fname"] = save_file
            
            audio_data_list.append(audio_data_dic)
    
    data_dic["data"] = audio_data_list
    data_list.append(data_dic)
#print(audio_data_list)

audio : C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/audio\audio1661283623_작업완료_한승연_2022_07_20.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/text\audio1661283623_작업완료_한승연_2022_07_20.txt
audio : C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/audio\audio1678210928_작업완료_한승연_2022_07_21.wav
text : C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/text\audio1678210928_작업완료_한승연_2022_07_21.txt


In [30]:
create_directory(base_path + '/result')
os.chdir(base_path + '/result')
#print(os.getcwd())
print(data_list)

#결과를 json파일로 저장
import json

with open("trim_audio_file.json", "wt") as json_file:
    for file_data in data_list:
        print(file_data)
        json.dump(file_data, json_file, ensure_ascii=False, indent = 4)

print("저장완료!")

[{'data': [{'transcript': '어 먼저 이 바쁘신 시간에 착수보고에 참석해주셔서 감사드리고 연일 날씨가 뭐 영하 십 도 씨를 기록하고 있습니다 추운 날씨에 건강하시고 또 특히 이 코로나에 유의하셨으면 좋겠습니다', 'fname': 'C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/save/audio1661283623_작업완료_한승연_2022_07_20/audio1661283623_작업완료_한승연_2022_07_20_0.wav'}, {'transcript': '그래서 이제 주로 이제 사업에 대한 내용과 그 감리의 적용 기준이라든지 감리를 수행할 체계 그리고 절차라든지 어 기타 사항 이런 순서로 진행을 하도록 하겠습니다', 'fname': 'C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/save/audio1661283623_작업완료_한승연_2022_07_20/audio1661283623_작업완료_한승연_2022_07_20_14.wav'}, {'transcript': '어 먼저 감리대상 사업명은 베트남 국가기술자격 정보시스템 구축 일 단계 감리 용역이고요', 'fname': 'C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/save/audio1661283623_작업완료_한승연_2022_07_20/audio1661283623_작업완료_한승연_2022_07_20_27.wav'}, {'transcript': '어 그렇습니까 네 잠깐만요 이제 보이시나요 그럼 다시 설명을 드리겠습니다 감리대상 사업명은 베트남 국가기술자격 정보시스템 구축 일 단계 감리 용역이고요', 'fname': 'C:/Users/hoy26/OneDrive/바탕 화면/dataset_0727/save/audio1661283623_작업완료_한승연_2022_07_20/audio1661283623_작업완료_한승연_2022_07_20_36.wav'}, {'transcript': '감리를 수행하는 근거는 한국산업